In [1]:
!pip install -U libsvm-official
!pip install -U liblinear-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for libsvm-official: filename=libsvm_official-3.31.0-cp310-cp310-linux_x86_64.whl size=125912 sha256=d92b27664d1e6c4df4f8eaea4c2ffbd3801a82ad3e989347b9bc5107f6bb80e5
  Stored in directory: /root/.cache/pip/wheels/93/04/ec/306a03478d811cf83c10665ca5e60231f47360a367e06d94d0
Successfully built libsvm-official
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for liblinear-official: filename=liblinear_official-2.46.1-cp310-cp310-linux_x86_64.whl size=146571 sha256=39cbd0c9d368069628ec46c09a5cad70d304e82f6b135766a8a6bdee30648380
  Stored in directory: /root/.cache/pip/wheels/d1/c4/f5/0f9664e5a5484cfe73e265aaa8eb83d8d51342760c979cb846
Successfully built li

In [2]:
import numpy as np
#from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from liblinear.liblinearutil import *
import pandas as pd

In [3]:
total_data=pd.read_csv("/content/breast-cancer_scale.txt", header=None, sep="[: ]+")
print("loaded data:")
print(total_data)
print()

total_data = total_data.iloc[:, ::2]
total_data.columns = range(total_data.shape[1])
print("modified data:")
print(total_data)
print()

train_indices = pd.read_csv("/content/breast-cancer-scale-train-indices.txt",header=None)
test_indices = pd.read_csv("/content/breast-cancer-scale-test-indices.txt",header=None)

# Select rows from data based on train_indices
train_data = total_data[total_data.index.isin(train_indices[0].values)]
# Select rows from data based on test_indices
test_data = total_data[total_data.index.isin(test_indices[0].values)]
print("checking train data:")
print(train_data)
print()

# Split train_data into X_train and y_train
X_train = train_data.drop(columns=train_data.columns[0]).values
y_train = train_data.iloc[:, 0].values.reshape(-1, 1)

# Split test_data into X_test and y_test
X_test = test_data.drop(columns=test_data.columns[0]).values
y_test = test_data.iloc[:, 0].values.reshape(-1, 1)


loaded data:
     0   1         2   3         4   5         6   7         8   9   ...  11  \
0     2   1 -0.860107   2 -0.111111   3 -1.000000   4 -1.000000   5  ...   6   
1     2   1 -0.859671   2 -0.111111   3 -0.333333   4 -0.333333   5  ...   6   
2     2   1 -0.857807   2 -0.555556   3 -1.000000   4 -1.000000   5  ...   6   
3     2   1 -0.857680   2  0.111111   3  0.555556   4  0.555556   5  ...   6   
4     2   1 -0.857569   2 -0.333333   3 -1.000000   4 -1.000000   5  ...   6   
..   ..  ..       ...  ..       ...  ..       ...  ..       ...  ..  ...  ..   
678   2   1 -0.893460   2 -0.555556   3 -1.000000   4 -1.000000   5  ...   6   
679   2   1 -0.883744   2 -0.777778   3 -1.000000   4 -1.000000   5  ...   6   
680   4   1 -0.876716   2 -0.111111   3  1.000000   4  1.000000   5  ...   6   
681   4   1 -0.875424   2 -0.333333   3  0.555556   4  0.111111   5  ...   6   
682   4   1 -0.875424   2 -0.333333   3  0.555556   4  0.555556   5  ...   6   

           12  13        1

<ipython-input-3-24068a0e20f7>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  total_data=pd.read_csv("/content/breast-cancer_scale.txt", header=None, sep="[: ]+")


# **Logictic Regression**

In [4]:
C_list = [0.1, 1.0, 10.0, 100.0, 1000.0]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-c {c} -s 0 -v 5"
    train_acc=train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()

print("Best accuracy = ",max(Cross_Validation_accuracy), "%")
best_c_val=C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))]
print("Best C value = ",best_c_val)

model = train(y_train.ravel(), X_train, f'-c {best_c_val} -s 0')
y_pred, metrics, _ = predict(y_test.ravel(), X_test, model)

Training C=0.1
Cross Validation Accuracy = 95.8%
Training Error= 4.200000000000003

Training C=1.0
Cross Validation Accuracy = 96.6%
Training Error= 3.4000000000000057

Training C=10.0
Cross Validation Accuracy = 96.6%
Training Error= 3.4000000000000057

Training C=100.0
Cross Validation Accuracy = 96.8%
Training Error= 3.200000000000003

Training C=1000.0
Cross Validation Accuracy = 97%
Training Error= 3.0

Best accuracy =  97.0 %
Best C value =  1000.0
Accuracy = 95.6044% (174/182) (classification)


In [5]:
print("Error Rate for Logistic Regression=",100-metrics[0])

Error Rate for Logistic Regression= 4.395604395604394


# **Linear SVM**


In [6]:
from libsvm.svmutil import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
C_list = [0.1, 1.0, 10.0, 100.0, 1000.0]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-c {c} -s 3 -v 5"
    train_acc=train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()

print("Best Accuracy = ",max(Cross_Validation_accuracy), "%")
best_c_val=C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))]
print("Best C value = ",best_c_val)

model = train(y_train.ravel(), X_train, f'-c {best_c_val} -s 3')
y_pred, metrics, _ = predict(y_test.ravel(), X_test, model)

Training C=0.1
Cross Validation Accuracy = 96.4%
Training Error= 3.6000000000000085

Training C=1.0
Cross Validation Accuracy = 97.4%
Training Error= 2.6000000000000085

Training C=10.0
Cross Validation Accuracy = 97%
Training Error= 3.0

Training C=100.0
Cross Validation Accuracy = 96.8%
Training Error= 3.200000000000003

Training C=1000.0
Cross Validation Accuracy = 89%
Training Error= 11.0

Best Accuracy =  97.39999999999999 %
Best C value =  1.0
Accuracy = 96.1538% (175/182) (classification)


In [12]:
print("Error Rate for Linear SVM=",100-metrics[0])

Error Rate for Linear SVM= 3.8461538461538396


# **Kernel SVM**

**Polynomial Kernel**

In [13]:
C_list = [0.1, 1.0, 10.0, 100.0, 1000.0]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-t 1 -c {c} -s 0 -v 5"
    train_acc=svm_train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()

print("Best Accuracy = ",max(Cross_Validation_accuracy), "%")
best_c_val=C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))]
print("Best C value = ",best_c_val)

model = svm_train(y_train.ravel(), X_train, f'-t 1 -c {best_c_val} -s 0')
y_pred, metrics, _ = svm_predict(y_test.ravel(), X_test, model)

Training C=0.1
Cross Validation Accuracy = 97%
Training Error= 3.0

Training C=1.0
Cross Validation Accuracy = 97.2%
Training Error= 2.799999999999997

Training C=10.0
Cross Validation Accuracy = 97.2%
Training Error= 2.799999999999997

Training C=100.0
Cross Validation Accuracy = 96%
Training Error= 4.0

Training C=1000.0
Cross Validation Accuracy = 94.6%
Training Error= 5.400000000000006

Best Accuracy =  97.2 %
Best C value =  1.0
Accuracy = 97.8022% (178/182) (classification)


In [14]:
print("Error Rate for SVM Kernel (Polynomial)=",100-metrics[0])

Error Rate for SVM Kernel (Polynomial)= 2.197802197802204


**RBF Kernel**

In [15]:
C_list = [0.1, 1.0, 10.0, 100.0, 1000.0]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-t 2 -c {c} -s 0 -v 5"
    train_acc=svm_train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()

print("Best Accuracy = ",max(Cross_Validation_accuracy), "%")
best_c_val=C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))]
print("Best C value = ",best_c_val)

model = svm_train(y_train.ravel(), X_train, f'-t 2 -c {best_c_val} -s 0')
y_pred, metrics, _ = svm_predict(y_test.ravel(), X_test, model)

Training C=0.1
Cross Validation Accuracy = 96.8%
Training Error= 3.200000000000003

Training C=1.0
Cross Validation Accuracy = 97.4%
Training Error= 2.6000000000000085

Training C=10.0
Cross Validation Accuracy = 97.2%
Training Error= 2.799999999999997

Training C=100.0
Cross Validation Accuracy = 96.2%
Training Error= 3.799999999999997

Training C=1000.0
Cross Validation Accuracy = 95.4%
Training Error= 4.6000000000000085

Best Accuracy =  97.39999999999999 %
Best C value =  1.0
Accuracy = 96.1538% (175/182) (classification)


In [16]:
print("Error Rate for SVM Kernel (RBF)=",100-metrics[0])

Error Rate for SVM Kernel (RBF)= 3.8461538461538396


## **SONAR DATA**

In [18]:
from sklearn.datasets import load_svmlight_file

In [19]:
X_total, y_total=load_svmlight_file("/content/sonar_scale.txt")
train_indices = np.loadtxt('/content/sonar-scale-train-indices.txt', dtype=int)
test_indices = np.loadtxt('/content/sonar-scale-test-indices.txt', dtype=int)
X_train =X_total[train_indices - 1]
y_train =y_total[train_indices -1]
X_test  =X_total[test_indices - 1]
y_test  =y_total[test_indices - 1]

# **LOGISTIC REGRESSION**

In [34]:
C_list = [0.1, 1.0, 10.0, 100.0, 1000.0]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-c {c} -s 0 -v 5"
    train_acc=train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()

print("Best accuracy = ",max(Cross_Validation_accuracy), "%")
best_c_val=C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))]
print("Best C value = ",best_c_val)

model = train(y_train.ravel(), X_train, f'-c {best_c_val} -s 0')
y_pred, metrics, _ = predict(y_test.ravel(), X_test, model)

Training C=0.1
Cross Validation Accuracy = 68.6667%
Training Error= 31.33333333333333

Training C=1.0
Cross Validation Accuracy = 73.3333%
Training Error= 26.66666666666667

Training C=10.0
Cross Validation Accuracy = 72.6667%
Training Error= 27.33333333333333

Training C=100.0
Cross Validation Accuracy = 74%
Training Error= 26.0

Training C=1000.0
Cross Validation Accuracy = 79.3333%
Training Error= 20.66666666666667

Best accuracy =  79.33333333333333 %
Best C value =  1000.0
Accuracy = 68.9655% (40/58) (classification)


In [35]:
print("Error Rate for Logistic Regression=",100-metrics[0])

Error Rate for Logistic Regression= 31.034482758620683


# **LINEAR SVM**

In [33]:
from libsvm.svmutil import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [24]:
C_list = [0.1, 1.0, 10.0, 100.0, 1000.0]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-c {c} -s 3 -v 5"
    train_acc=train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()

print("Best Accuracy = ",max(Cross_Validation_accuracy), "%")
best_c_val=C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))]
print("Best C value = ",best_c_val)

model = train(y_train.ravel(), X_train, f'-c {best_c_val} -s 3')
y_pred, metrics, _ = predict(y_test.ravel(), X_test, model)

Training C=0.1
Cross Validation Accuracy = 71.3333%
Training Error= 28.666666666666657

Training C=1.0
Cross Validation Accuracy = 75.3333%
Training Error= 24.66666666666667

Training C=10.0
Cross Validation Accuracy = 72.6667%
Training Error= 27.33333333333333

Training C=100.0
Cross Validation Accuracy = 76%
Training Error= 24.0

Training C=1000.0
Cross Validation Accuracy = 66.6667%
Training Error= 33.33333333333334

Best Accuracy =  76.0 %
Best C value =  100.0
Accuracy = 70.6897% (41/58) (classification)


In [25]:
print("Error Rate for SVM Linear=",100-metrics[0])

Error Rate for SVM Linear= 29.310344827586206


# **KERNEL SVM**

**POLYNOMIAL KERNEL**

In [31]:
C_list = [0.1, 1.0, 10.0, 100.0, 1000.0]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-t 1 -c {c} -s 0 -v 5"
    train_acc=svm_train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()

print("Best Accuracy = ",max(Cross_Validation_accuracy), "%")
best_c_val=C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))]
print("Best C value = ",best_c_val)

model = svm_train(y_train.ravel(), X_train, f'-t 1 -c {best_c_val} -s 0')
y_pred, metrics, _ = svm_predict(y_test.ravel(), X_test, model)

Training C=0.1
Cross Validation Accuracy = 56%
Training Error= 43.99999999999999

Training C=1.0
Cross Validation Accuracy = 56.6667%
Training Error= 43.333333333333336

Training C=10.0
Cross Validation Accuracy = 75.3333%
Training Error= 24.66666666666667

Training C=100.0
Cross Validation Accuracy = 87.3333%
Training Error= 12.666666666666671

Training C=1000.0
Cross Validation Accuracy = 81.3333%
Training Error= 18.66666666666667

Best Accuracy =  87.33333333333333 %
Best C value =  100.0
Accuracy = 87.931% (51/58) (classification)


In [32]:
print("Error Rate for SVM Kernel (Polynomial)=",100-metrics[0])

Error Rate for SVM Kernel (Polynomial)= 12.06896551724138


**RBF KERNEL**

In [29]:
C_list = [0.1, 1.0, 10.0, 100.0, 1000.0]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-t 2 -c {c} -s 0 -v 5"
    train_acc=svm_train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()

print("Best Accuracy = ",max(Cross_Validation_accuracy), "%")
best_c_val=C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))]
print("Best C value = ",best_c_val)

model = svm_train(y_train.ravel(), X_train, f'-t 2 -c {best_c_val} -s 0')
y_pred, metrics, _ = svm_predict(y_test.ravel(), X_test, model)

Training C=0.1
Cross Validation Accuracy = 56%
Training Error= 43.99999999999999

Training C=1.0
Cross Validation Accuracy = 78.6667%
Training Error= 21.333333333333343

Training C=10.0
Cross Validation Accuracy = 80%
Training Error= 20.0

Training C=100.0
Cross Validation Accuracy = 83.3333%
Training Error= 16.666666666666657

Training C=1000.0
Cross Validation Accuracy = 81.3333%
Training Error= 18.66666666666667

Best Accuracy =  83.33333333333334 %
Best C value =  100.0
Accuracy = 86.2069% (50/58) (classification)


In [30]:
print("Error Rate for SVM Kernel (RBF)=",100-metrics[0])

Error Rate for SVM Kernel (RBF)= 13.793103448275872


# **ACCURACY COMPARISON FOR BREAST CANCER DATA**


\begin{array}{|c|c|c|c|c|} \hline
Factors & Train/Test Metric & Logistic Regression & Linear SVM & Kernel SVM Polynomial & Kernel SVM RBF\\ \hline
Classification Accuracy & Train Accuracy & 97.\% & 97.4\% & 97.2\% & 97.4\%\\ \hline
Classification Accuracy & Test Accuracy & 95.60\% & 96.15\% & 97.80\% & 96.15\%\\ \hline
Best C val & using Cross Validation & 1000.0 & 1.0 & 1.0 & 1.0\\ \hline
\end{array}

## **ACCURACY COMPARISON FOR SONAR DATA**


\begin{array}{|c|c|c|c|c|} \hline
Factors & Train/Test Metric & Logistic Regression & Linear SVM & Kernel SVM Polynomial & Kernel SVM RBF\\ \hline
Classification Accuracy & Train Accuracy & 79.33\%\% & 76.0\% & 87.33\% & 83.33\%\\ \hline
Classification Accuracy & Test Accuracy & 68.96\% & 70.69\% & 87.93\% & 86.21\%\\ \hline
Best C val & using Cross Validation & 1000 & 100 & 100 & 100\\ \hline
\end{array}

# **ERROR RATE COMPARISON OF TEST DATA (as per question)**


\begin{array}{|c|c|c|c|c|} \hline
Data & Logistic Regression & Linear SVM & Kernel SVM Polynomial & Kernel SVM RBF\\ \hline
Breast Cancer Data & 4.39\% & 3.85\% & 2.19\% & 3.85\%\\ \hline
Sonar Data & 31.03\% & 29.31\% & 12.07\% & 13.79\%\\ \hline
\end{array}

# ***OBSERVATION ANALYSIS:***

**We see for test accuracies-Kernel SVM polynomial behaves the best, then kernel SVM RBF. (In case of Sonar Data, both kernel SVMs behave almost similarly) 3rd best is linear SVM and last is Logistic regression.**

In the case of these four models - Logistic Regression, Linear SVM, Kernel SVM Polynomial, and Kernel SVM RBF - they are all used for binary classification problems, but they have different ways of separating the data.

**Logistic Regression** is a linear model that tries to find a decision boundary that separates the two classes. It works well when the data is linearly separable, but it can struggle with more complex patterns in the data.

**Linear SVM** is also a linear model, but it tries to find the decision boundary that maximizes the margin between the two classes. It works well when the data is linearly separable, and** it is less affected by outliers than Logistic Regression.**

**Kernel SVM Polynomial and Kernel SVM RBF** are non-linear models that use kernel functions to map the data into a higher-dimensional space where the classes can be separated by a hyperplane. Kernel SVM Polynomial uses a polynomial kernel function, while Kernel SVM RBF uses a radial basis function kernel. These models are more flexible than Logistic Regression and Linear SVM, and they can handle complex non-linear decision boundaries.

In general, **Kernel SVM Polynomial and Kernel SVM RBF are more accurate than Logistic Regression and Linear SVM when the data has non-linear decision boundaries**. This is because they can map the data into a higher-dimensional space where it becomes easier to separate the classes. Among the two Kernel SVM models, **Kernel SVM Polynomial is usually more accurate than Kernel SVM RBF when the degree of the polynomial is chosen appropriately(here default value is 3)**, because it can capture more complex decision boundaries. However, Kernel SVM Polynomial can be more sensitive to overfitting if the degree of the polynomial is too high.

**Linear SVM can be more accurate than Logistic Regression when the data has a clear margin between the two classes**, and it can be more robust to outliers. However, Logistic Regression can be more accurate than Linear SVM when the data is not linearly separable, but the decision boundary is still close to linear.

# ***Overall,from the above observation, my CONCLUSION is as followed***:
1.   The dataset given has non-linear decision boundary since kernel SVMs give better accuracies
2.   The data has clear margin between two classes since Linear SVM gives better accuracy 








